In [ ]:
import moo_benchmarks as mb
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_benchmark
import controller
import performance_measure as pm
import pymop

import plotly.express as px
import plotly.graph_objects as go
import time
import numpy as np

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging

# Blackboard Verification using MOO Benchmarks

In this notebook, we utilize the blackboard to find the Pareto front for a few common multi-objective optimization problems.

## Schaffer Functions

### Schaffer Function 1
We start by examining a multi-objective test function known as the Schaffer Functions \cite{Schaffer1984}.
Schaffer function 1 has two objective functions ($f_1, f_2$), which takes the form:

\begin{equation}
f_1 = x^2 \\
f_2 = (x-2)^2 \\
-A \leq x \leq A \\
\end{equation}

### Schaffer Function 2

Where A the greater the value for A, the more difficult the problem is to solve for an optimization algorithm.
Schaffer function 2 examines an objective space with a discontinuity by examining the following equation:

\begin{equation}
f_{1} = \begin{cases}
-x & \text{if $x \leq 1$;}\\
x-2 & \text{if $1 < x \leq 3$;}\\
4 - x & \text{if $3 < x \leq 4$;}\\
x-4& \text{if $ x > 4$.}
\end{cases}\\
f_2 = (x-5)^2 \\
-5 \leq x \leq 10
\end{equation}



In [ ]:
model = 'sf1'
objs = {'f1': {'ll':0, 'ul':4, 'goal':'lt', 'variable type': float},
        'f2': {'ll':0, 'ul':4, 'goal':'lt', 'variable type': float}}
dvs =  {'x1':  {'ll':-10, 'ul':10, 'variable type': float}}
const = {}

kas = {'ka_rp_explore': karp.KaGlobal, 
       'ka_rp_hc': karp.KaLocalHC,
       'ka_rp_pert': karp.KaLocal,
       'ka_rp_ga': karp.KaGA,
       'ka_br_lvl3': kabr.KaBr_lvl3,
       'ka_br_lvl2': kabr.KaBr_lvl2,
       'ka_br_lvl1': kabr.KaBr_lvl1}

bb_controller = controller.Controller(bb_name=model, 
                                      bb_type=bb_benchmark.BenchmarkBB, 
                                      ka=kas, archive='{}_benchmark_2'.format(model),
                                      objectives=objs, 
                                      design_variables=dvs,
                                      benchmark=model, 
                                      agent_wait_time=60*30, 
                                      plot_progress=True,
                                      progress_rate=20)
bb = bb_controller.bb
bb.set_attr(num_calls=10)
ka = bb.get_attr('_proxy_server')
ka2 = ka.proxy('ka_rp_hc')
ka2.set_attr(step_limit=10)

br1 = ka.proxy('ka_br_lvl1')
br1.set_attr(total_pf_size = 200)
br1.set_attr(pareto_sorter='dci')
br1.set_attr(dci_div={'f1': 40, 'f2': 40})
ka4 = ka.proxy('ka_rp_ga')    
ka4.set_attr(crossover_type='linear crossover')

bb.set_attr(hv_convergence=1E-5)
bb_controller.run_single_agent_bb()
#bb_controller.shutdown()

The resulting Pareto front for the Schaffer fuction 1 can be seen below. This utilizes a hypervolume convergence criteria of 1E-6 over the past 25 solutions to determine a stopping criteria. Below the Pareto front, we see that hypbervolume indicator for each trigger event.

![SF1 Pareto Front](../doc/figs/sf1_pf.png)
![SF1 Hypervolume](../doc/figs/sf1_hv.png)

In [ ]:
model = 'sf2'
objs = {'f1': {'ll':-1, 'ul':1, 'goal':'lt', 'variable type': float},
        'f2': {'ll':0, 'ul':16, 'goal':'lt', 'variable type': float}}
dvs =  {'x1':  {'ll':-5, 'ul':10, 'variable type': float}}

kas = {'ka_rp_explore': karp.KaGlobal, 
       'ka_rp_hc': karp.KaLocalHC,
       'ka_rp_pert': karp.KaLocal,
       'ka_rp_ga': karp.KaGA,
       'ka_br_lvl3': kabr.KaBr_lvl3,
       'ka_br_lvl2': kabr.KaBr_lvl2,
       'ka_br_lvl1': kabr.KaBr_lvl1}
bb_controller = controller.Controller(bb_name=model, 
                                      bb_type=bb_benchmark.BenchmarkBB, 
                                      ka=kas, archive='{}_benchmark_2'.format(model),
                                      objectives=objs, 
                                      design_variables=dvs,
                                      benchmark=model, 
                                      agent_wait_time=60*30, 
                                      plot_progress=True,
                                      progress_rate=20)
bb = bb_controller.bb
bb.set_attr(num_calls=10)
ka = bb.get_attr('_proxy_server')
ka2 = ka.proxy('ka_rp_hc')
ka2.set_attr(step_limit=10)

br1 = ka.proxy('ka_br_lvl1')
br1.set_attr(total_pf_size = 200)
br1.set_attr(pareto_sorter='dci')
br1.set_attr(dci_div={'f1': 50, 'f2': 80})
ka4 = ka.proxy('ka_rp_ga')    
ka4.set_attr(crossover_type='linear crossover')

bb.set_attr(hv_convergence=1E-5)
bb_controller.run_single_agent_bb()
bb_controller.shutdown()

The resulting Pareto front for the Schaffer fuction 2 can be seen below. This utilizes a hypervolume convergence criteria of 1E-6 over the past 25 solutions to determine a stopping criteria. Below the Pareto front, we see that hypbervolume indicator for each trigger event.

![SF2 Pareto Front](../doc/figs/sf2_pf.png)
![SF2 Hypervolume](../doc/figs/sf2_hv.png)

## Zitlzer-Deb-Thiele (ZDT) Functions

We next explore the ZDT functions which take the general form below \cite{Deb2002}:

\begin{equation}
\text{min  } f_1(x) = x_1 \\
\text{min  } f_2(x) = g(x)h(f_x(x),g(x))
\end{equation}

### ZDT1

We start by examining ZDT1 where we express $f_1$ and $f_2$ below:

\begin{equation}
\text{min  } f_1(x) = x_1 \\
\text{min  } f_2(x) = g(x)h(f_1(x),g(x)) \\
g(x) = 1 + \frac{9}{n-1}\sum_{i=2}^{n}x_i \\
h(f_1,g)=1-\sqrt{f_1/g}
\end{equation}


In [ ]:
model = 'zdt1'
objs = {}
dvs =  {}
dvi_ = {}
for x in range(2):
    dvs['x{}'.format(x)] = {'ll':0, 'ul':1, 'variable type': float}

for x in range(2):
    objs['f{}'.format(x)] = {'ll':0, 'ul':1, 'goal':'lt', 'variable type': float}
    dvi_['f{}'.format(x)] = 50
    
kas = {'ka_rp_explore': karp.KaGlobal, 
       'ka_rp_hc': karp.KaGlobal,
       'ka_rp_pert': karp.KaLocal,
       'ka_rp_ga': karp.KaGA,
       'ka_br_lvl3': kabr.KaBr_lvl3,
       'ka_br_lvl2': kabr.KaBr_lvl2,
       'ka_br_lvl1': kabr.KaBr_lvl1}
bb_controller = controller.Controller(bb_name=model, 
                                      bb_type=bb_benchmark.BenchmarkBB, 
                                      ka=kas, archive='{}_benchmark_2'.format(model),
                                      objectives=objs, 
                                      design_variables=dvs,
                                      benchmark=model, 
                                      agent_wait_time=60*30, 
                                      plot_progress=True,
                                      progress_rate=20)
bb = bb_controller.bb
bb.set_attr(num_calls=10)
ka = bb.get_attr('_proxy_server')
ka2 = ka.proxy('ka_rp_hc')
ka2.set_attr(step_limit=10)

br1 = ka.proxy('ka_br_lvl1')
br1.set_attr(total_pf_size = 200)
br1.set_attr(pareto_sorter='dci')
br1.set_attr(dci_div=dvi_)
ka4 = ka.proxy('ka_rp_ga')    
ka4.set_attr(crossover_type='linear crossover')

bb.set_attr(hv_convergence=1E-6)
bb_controller.run_single_agent_bb()
bb_controller.shutdown()

The resulting Pareto front for the ZDT fuction 2 can be seen below along with the hypervolume indicator for each trigger event.

![ZDT1 Pareto Front](../doc/figs/zdt_pf.png)
![ZDT1 Hypervolume](../doc/figs/zdt_hv.png)

In [ ]:
from pymop.factory import get_problem, get_uniform_weights
ref_dirs = get_uniform_weights(100, 2)
pf = get_problem("dtlz1", n_var=2, n_obj=2).pareto_front(ref_dirs)
pf = get_problem("zdt1").pareto_front(n_pareto_points=100)
dict_1 = {}
for num, soln in enumerate(pf):
    dict_1[num] = {'f{}'.format(x): y for x,y in enumerate(soln)}


fig1 = px.scatter(x=pf[:,0], y=pf[:,1], labels={'x':'f1', 'y': 'f2',})
#fig2 = px.scatter_3d(x=pf[:,0], y=pf[:,1], z=pf[:,2], labels={'x':'f1', 'y': 'f2', 'z': 'f3'})
fig1.show()
#fig2.show()

# References

(<a id="cit-Schaffer1984" href="#call-Schaffer1984">Schaffer, 1984</a>) James David Schaffer, ``_Some Experiments in Machine Learning Using Vector Evaluated Genetic Algorithms (Artificial Intelligence, Optimization, Adaptation, Pattern Recognition)_'',  1984.

(<a id="cit-Deb2002" href="#call-Deb2002">Deb, Thiele <em>et al.</em>, 2002</a>) K. Deb, L. Thiele, M. Laumanns <em>et al.</em>, ``_Scalable multi-objective optimization test problems_'', Proceedings of the 2002 Congress on Evolutionary Computation. CEC'02 (Cat. No.02TH8600),  2002.



In [ ]:

N=25
benchmark = 'sf1'
benchmark2 = 'sf2'

bm = mb.optimization_test_functions(benchmark)
bm2 = mb.optimization_test_functions(benchmark2)

x1 = np.linspace(0, 1, 2*N, endpoint=True)
x2 = list(np.linspace(1, 2, N, endpoint=True))
x3 = list(np.linspace(4, 5, N, endpoint=True))
x2 += x3

f1 = []
f2 = []
f3 = []
f4 = []

for i,j in zip(x1,x2):
    sf1 = bm.predict(benchmark, [i])
    sf2 = bm2.predict(benchmark2, [j])
    f1.append(sf1[0])
    f2.append(sf1[1])
    f3.append(sf2[0])
    f4.append(sf2[1])

fig1 = px.scatter(x=f1, y=f2, color=x1, labels={'x':'f1', 'y': 'f2', 'color': 'x'})
fig1.show()
fig1 = px.scatter(x=f3, y=f4, color=x2, labels={'x':'f1', 'y': 'f2', 'color': 'x'})
fig1.show()

fig1 = px.scatter(x=f1, y=f2, labels={'x':'f1', 'y': 'f2'})
fig1.show()
fig1 = px.scatter(x=f3, y=f4, labels={'x':'f1', 'y': 'f2'})
fig1.show()
